In [60]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
import lightgbm as lgb 
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [122]:
csv_path = r"C:\Users\cirri\OneDrive\Desktop\Cienciadedatos\DatasetFinal.csv"
df = pd.read_csv(csv_path)

In [123]:
# Seleccionar solo las columnas necesarias
columnas_seleccionadas = [
    'barrio', 
    'ambientes', 
    'habitaciones', 
    'baños', 
    'surface_total', 
    'surface_covered', 
    'precio', 
    'comuna',
    'zona'
]
df = df[columnas_seleccionadas]

## Variables de interaccion X Zona

In [124]:
# Asegurarnos de que df existe
if 'df' not in globals():
    raise NameError('No se encuentra el DataFrame `df`. Ejecuta la celda de carga antes de correr esto.')

# Detectar columna de zona (case-insensitive)
zona_candidates = [c for c in df.columns if c.lower() in ('zona','zone')]
if not zona_candidates:
    raise KeyError('No se encontró la columna `zona` en el dataset. Asegúrate de tener una columna con el nombre `zona` que contenga Norte/Sur/Centro/Oeste.')

zona_col = zona_candidates[0]
# Normalizar valores de zona comunes
df[zona_col] = df[zona_col].astype(str)
df[zona_col] = df[zona_col].replace({
    'Cento/Oeste': 'Centro/Oeste',
    'Centro Oeste': 'Centro/Oeste',
    'CentroOeste': 'Centro/Oeste',
    'Centro': 'Centro/Oeste'  # en caso de variantes
})

zones = ['Norte', 'Sur', 'Centro/Oeste']

# Definición de posibles nombres de columnas para cada variable y nombres cortos para las interacciones
candidates = {
    'habitaciones': (['habitaciones', 'rooms', 'bedrooms', 'habitacion'], 'hab'),
    'banos': (['baños', 'banos', 'bathrooms', 'bathroom'], 'banos'),
    'ambientes': (['ambientes', 'ambiente', 'rooms_total', 'amb'], 'amb'),
    'superficie_cubierta': (['surface_covered', 'surface_covered_m2', 'surface_covered_in_m2', 'superficie_cubierta', 'surface_covered', 'surface_coveres', 'surface_total'], 'sup_cub'),
    'superficie_total': (['surface_total', 'surface_total_m2', 'superficie_total', 'totalsurface', 'surface_total_m'], 'sup_tot')
}

found = {}
for key, (opts, short) in candidates.items():
    for o in opts:
        if o in df.columns:
            found[key] = {'col': o, 'short': short}
            break

if not found:
    raise KeyError('No se encontraron ninguna de las columnas objetivo (habitaciones/baños/ambientes/superficie) en el dataset. Revisa los nombres de columnas.')

# Convertir a numérico donde corresponda
for k, info in found.items():
    col = info['col']
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Crear interacciones: variable * zona (sin crear flags separados ya que 'zona' tendrá OneHotEncoding)
# Las interacciones son: habitaciones*zona, baños*zona, ambientes*zona, superficie_cubierta*zona, superficie_total*zona
created = []
for k, info in found.items():
    col = info['col']
    short = info['short']
    for z in zones:
        zone_tag = z.lower().replace('/', '_').replace(' ', '_')
        new_col = f"{short}_x_{zone_tag}"
        df[new_col] = df[col] * (df[zona_col] == z).astype(int)
        created.append(new_col)

print(f'Se crearon {len(created)} variables de interacción:')
print(created)

Se crearon 15 variables de interacción:
['hab_x_norte', 'hab_x_sur', 'hab_x_centro_oeste', 'banos_x_norte', 'banos_x_sur', 'banos_x_centro_oeste', 'amb_x_norte', 'amb_x_sur', 'amb_x_centro_oeste', 'sup_cub_x_norte', 'sup_cub_x_sur', 'sup_cub_x_centro_oeste', 'sup_tot_x_norte', 'sup_tot_x_sur', 'sup_tot_x_centro_oeste']


## Variables de interaccion por Comuna

In [125]:
# Verificar que el DataFrame existe
if 'df' not in globals():
    raise NameError('No se encuentra el DataFrame `df`. Ejecuta la celda de carga antes de correr esto.')

# Detectar columna de comuna (case-insensitive)
comuna_candidates = [c for c in df.columns if 'comuna' in c.lower() or c.lower() in ('comuna','district','commune','comuna_id','comuna_num')]
if not comuna_candidates:
    raise KeyError('No se encontró la columna `comuna` en el dataset. Revisa los nombres de columna.')

comuna_col = comuna_candidates[0]

# Reutilizar mapeo `found` si existe (detecta las columnas objetivo), sino detectarlas aquí
if 'found' in globals() and found:
    found_used = found
else:
    candidates_obj = {
        'habitaciones': (['habitaciones', 'rooms', 'bedrooms', 'habitacion'], 'hab'),
        'banos': (['baños', 'banos', 'bathrooms', 'bathroom'], 'banos'),
        'ambientes': (['ambientes', 'ambiente', 'rooms_total', 'amb'], 'amb'),
        'superficie_cubierta': (['surface_covered', 'surface_covered_m2', 'surface_covered_in_m2', 'superficie_cubierta', 'surface_coveres'], 'sup_cub'),
        'superficie_total': (['surface_total', 'surface_total_m2', 'superficie_total', 'totalsurface', 'surface_total_m'], 'sup_tot')
    }
    found_used = {}
    for key, (opts, short) in candidates_obj.items():
        for o in opts:
            if o in df.columns:
                found_used[key] = {'col': o, 'short': short}
                break

if not found_used:
    raise KeyError('No se encontraron las columnas objetivo (habitaciones/baños/ambientes/superficies).')

# Asegurarnos que las columnas objetivo sean numéricas
for k, info in found_used.items():
    col = info['col']
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Contar comunas y filtrar las que tienen suficientes observaciones
comuna_counts = df[comuna_col].value_counts()
min_obs = 5  # ajustable
comunas_validas = comuna_counts[comuna_counts >= min_obs].index.tolist()

created_comuna = []
for k, info in found_used.items():
    col = info['col']
    short = info['short']
    for comuna in comunas_validas:
        # Normalizar etiqueta de comuna para el nombre de columna
        comuna_tag = str(comuna).lower().replace(' ', '_').replace('/', '_').replace('-', '_')
        comuna_tag = ''.join(c for c in comuna_tag if c.isalnum() or c == '_')
        new_col = f"{short}_x_comuna_{comuna_tag}"
        df[new_col] = df[col] * (df[comuna_col] == comuna).astype(int)
        created_comuna.append(new_col)

print(f'\n Se crearon {len(created_comuna)} variables de interacción por comuna')
print(created_comuna)


 Se crearon 75 variables de interacción por comuna
['hab_x_comuna_14', 'hab_x_comuna_13', 'hab_x_comuna_2', 'hab_x_comuna_5', 'hab_x_comuna_15', 'hab_x_comuna_1', 'hab_x_comuna_6', 'hab_x_comuna_3', 'hab_x_comuna_12', 'hab_x_comuna_4', 'hab_x_comuna_7', 'hab_x_comuna_11', 'hab_x_comuna_10', 'hab_x_comuna_9', 'hab_x_comuna_8', 'banos_x_comuna_14', 'banos_x_comuna_13', 'banos_x_comuna_2', 'banos_x_comuna_5', 'banos_x_comuna_15', 'banos_x_comuna_1', 'banos_x_comuna_6', 'banos_x_comuna_3', 'banos_x_comuna_12', 'banos_x_comuna_4', 'banos_x_comuna_7', 'banos_x_comuna_11', 'banos_x_comuna_10', 'banos_x_comuna_9', 'banos_x_comuna_8', 'amb_x_comuna_14', 'amb_x_comuna_13', 'amb_x_comuna_2', 'amb_x_comuna_5', 'amb_x_comuna_15', 'amb_x_comuna_1', 'amb_x_comuna_6', 'amb_x_comuna_3', 'amb_x_comuna_12', 'amb_x_comuna_4', 'amb_x_comuna_7', 'amb_x_comuna_11', 'amb_x_comuna_10', 'amb_x_comuna_9', 'amb_x_comuna_8', 'sup_cub_x_comuna_14', 'sup_cub_x_comuna_13', 'sup_cub_x_comuna_2', 'sup_cub_x_comuna_5',

## Variables de interaccion por barrio


In [126]:
# Crear variables de interacción entre variables clave y los barrios
# Variables objetivo: habitaciones, baños, ambientes, superficie cubierta, superficie total
import pandas as pd

# Asegurarnos de que df existe
if 'df' not in globals():
    raise NameError('No se encuentra el DataFrame `df`. Ejecuta la celda de carga antes de correr esto.')

# Detectar columna de barrio (case-insensitive)
barrio_candidates = [c for c in df.columns if c.lower() in ('barrio', 'neighborhood', 'neighbourhood')]
if not barrio_candidates:
    raise KeyError('No se encontró la columna `barrio` en el dataset.')

barrio_col = barrio_candidates[0]

# Obtener la lista de barrios únicos (sin nulos)
barrios = df[barrio_col].dropna().unique()
print(f"Se encontraron {len(barrios)} barrios únicos:")
print(f"Primeros 10 barrios: {list(barrios[:10])}")

# Las variables ya fueron definidas anteriormente, reutilizamos la misma info
# Si no existen, las buscamos de nuevo
if 'found' not in globals() or not found:
    candidates_barrio = {
        'habitaciones': (['habitaciones', 'rooms', 'bedrooms', 'habitacion'], 'hab'),
        'banos': (['baños', 'banos', 'bathrooms', 'bathroom'], 'banos'),
        'ambientes': (['ambientes', 'ambiente', 'rooms_total', 'amb'], 'amb'),
        'superficie_cubierta': (['surface_covered', 'surface_covered_m2', 'surface_covered_in_m2', 'superficie_cubierta', 'surface_covered', 'surface_coveres', 'surface_total'], 'sup_cub'),
        'superficie_total': (['surface_total', 'surface_total_m2', 'superficie_total', 'totalsurface', 'surface_total_m'], 'sup_tot')
    }
    
    found = {}
    for key, (opts, short) in candidates_barrio.items():
        for o in opts:
            if o in df.columns:
                found[key] = {'col': o, 'short': short}
                break

if not found:
    raise KeyError('No se encontraron ninguna de las columnas objetivo (habitaciones/baños/ambientes/superficie).')

# Asegurarnos que las columnas estén en formato numérico
for k, info in found.items():
    col = info['col']
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Crear interacciones: variable * barrio
# ADVERTENCIA: Esto creará muchas columnas (4 variables × número de barrios)
# Recomendación: Filtrar solo barrios con suficientes observaciones
barrio_counts = df[barrio_col].value_counts()
min_obs = 5  # Mínimo de observaciones por barrio para crear interacciones
barrios_validos = barrio_counts[barrio_counts >= min_obs].index.tolist()

created_barrio = []
for k, info in found.items():
    col = info['col']
    short = info['short']
    for barrio in barrios_validos:
        # Limpiar nombre del barrio para usarlo como sufijo de columna
        barrio_tag = str(barrio).lower().replace(' ', '_').replace('/', '_').replace('-', '_')
        barrio_tag = ''.join(c for c in barrio_tag if c.isalnum() or c == '_')  # Solo alfanuméricos y _
        new_col = f"{short}_x_barrio_{barrio_tag}"
        df[new_col] = df[col] * (df[barrio_col] == barrio).astype(int)
        created_barrio.append(new_col)

print(f'\n Se crearon {len(created_barrio)} variables de interacción por barrio')
print(created_barrio)

Se encontraron 48 barrios únicos:
Primeros 10 barrios: ['San Cristobal', 'Boedo', 'Almagro', 'Flores', 'Nuñez', 'Palermo', 'Belgrano', 'Floresta', 'Recoleta', 'Balvanera']


C:\Users\cirri\AppData\Local\Temp\ipykernel_3768\1235078302.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col] * (df[barrio_col] == barrio).astype(int)
C:\Users\cirri\AppData\Local\Temp\ipykernel_3768\1235078302.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col] * (df[barrio_col] == barrio).astype(int)
C:\Users\cirri\AppData\Local\Temp\ipykernel_3768\1235078302.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time


 Se crearon 240 variables de interacción por barrio
['hab_x_barrio_palermo', 'hab_x_barrio_recoleta', 'hab_x_barrio_almagro', 'hab_x_barrio_caballito', 'hab_x_barrio_belgrano', 'hab_x_barrio_villa_crespo', 'hab_x_barrio_balvanera', 'hab_x_barrio_villa_urquiza', 'hab_x_barrio_flores', 'hab_x_barrio_san_telmo', 'hab_x_barrio_nuñez', 'hab_x_barrio_puerto_madero', 'hab_x_barrio_colegiales', 'hab_x_barrio_san_nicolás', 'hab_x_barrio_san_cristobal', 'hab_x_barrio_barracas', 'hab_x_barrio_villa_del_parque', 'hab_x_barrio_constitución', 'hab_x_barrio_villa_devoto', 'hab_x_barrio_saavedra', 'hab_x_barrio_retiro', 'hab_x_barrio_liniers', 'hab_x_barrio_villa_luro', 'hab_x_barrio_monserrat', 'hab_x_barrio_paternal', 'hab_x_barrio_boedo', 'hab_x_barrio_parque_patricios', 'hab_x_barrio_floresta', 'hab_x_barrio_villa_lugano', 'hab_x_barrio_boca', 'hab_x_barrio_parque_chacabuco', 'hab_x_barrio_chacarita', 'hab_x_barrio_coghlan', 'hab_x_barrio_villa_pueyrredón', 'hab_x_barrio_monte_castro', 'hab_x_bar

C:\Users\cirri\AppData\Local\Temp\ipykernel_3768\1235078302.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col] * (df[barrio_col] == barrio).astype(int)
C:\Users\cirri\AppData\Local\Temp\ipykernel_3768\1235078302.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df[col] * (df[barrio_col] == barrio).astype(int)
C:\Users\cirri\AppData\Local\Temp\ipykernel_3768\1235078302.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

Chequeamos que el dataframe tenga las nuevas variables

In [127]:
print("Número de columnas:", df.shape[1])

Número de columnas: 339


## Eliminacion de Nan para surface_total y surface_covered

In [128]:
# Comprobar columnas
for c in ('surface_total', 'surface_covered'):
    if c not in df.columns:
        raise KeyError(f"Columna '{c}' no encontrada en df")

is_nan_tot = df['surface_total'].isna()
is_nan_cov = df['surface_covered'].isna()

n_tot = is_nan_tot.sum()
n_cov = is_nan_cov.sum()
n_both = (is_nan_tot & is_nan_cov).sum()
n_only_tot = (is_nan_tot & ~is_nan_cov).sum()
n_only_cov = (is_nan_cov & ~is_nan_tot).sum()

print(f"Filas totales: {len(df)}")
print(f"NaN en surface_total: {n_tot}")
print(f"NaN en surface_covered: {n_cov}")
print(f"NaN en ambas columnas: {n_both}")
print(f"NaN solo en surface_total: {n_only_tot}")
print(f"NaN solo en surface_covered: {n_only_cov}")
# Comprobar columnas
for c in ('surface_total', 'surface_covered'):
    if c not in df.columns:
        raise KeyError(f"Columna '{c}' no encontrada en df")

is_nan_tot = df['surface_total'].isna()
is_nan_cov = df['surface_covered'].isna()

Filas totales: 37429
NaN en surface_total: 4027
NaN en surface_covered: 4182
NaN en ambas columnas: 3950
NaN solo en surface_total: 77
NaN solo en surface_covered: 232


## LAS CELDAS CON NAN EN AMBAS SERAN ELIMINADAS, Y LAS CELDAS CON SOLO UN NAN SE RELLENARA CON EL VALOR DEL NO NAN

In [129]:
# Rellenar NaNs entre surface_total y surface_covered y eliminar filas donde ambas sean NaN
from IPython.display import display

# Comprobar que las columnas existen
for c in ('surface_total', 'surface_covered'):
    if c not in df.columns:
        raise KeyError(f"Columna '{c}' no encontrada en df")

# Máscaras iniciales
is_tot_na = df['surface_total'].isna()
is_cov_na = df['surface_covered'].isna()

n_tot = is_tot_na.sum()
n_cov = is_cov_na.sum()
n_both = (is_tot_na & is_cov_na).sum()
n_only_tot = (is_tot_na & ~is_cov_na).sum()
n_only_cov = (is_cov_na & ~is_tot_na).sum()

print("Antes de rellenar:")
print(f"  Filas totales: {len(df)}")
print(f"  NaN en surface_total: {n_tot}")
print(f"  NaN en surface_covered: {n_cov}")
print(f"  NaN en ambas columnas: {n_both}")
print(f"  NaN solo en surface_total: {n_only_tot}")
print(f"  NaN solo en surface_covered: {n_only_cov}")

# Mostrar algunos ejemplos
cols_show = [c for c in ['barrio','comuna','zona','precio','surface_total','surface_covered'] if c in df.columns]
if n_only_tot:
    print('\nEjemplos (NaN solo en surface_total):')
    display(df.loc[is_tot_na & ~is_cov_na, cols_show].head(10))
if n_only_cov:
    print('\nEjemplos (NaN solo en surface_covered):')
    display(df.loc[is_cov_na & ~is_tot_na, cols_show].head(10))
if n_both:
    print('\nEjemplos (NaN en ambas):')
    display(df.loc[is_tot_na & is_cov_na, cols_show].head(10))

# Rellenar: si una de las dos tiene valor, copiar ese valor a la otra
mask_fill_tot = is_tot_na & ~is_cov_na
mask_fill_cov = is_cov_na & ~is_tot_na

# Copiar valores (operación en bloque)
df.loc[mask_fill_tot, 'surface_total'] = df.loc[mask_fill_tot, 'surface_covered']
df.loc[mask_fill_cov, 'surface_covered'] = df.loc[mask_fill_cov, 'surface_total']

# Eliminar filas donde ambas siguen siendo NaN
both_na = df['surface_total'].isna() & df['surface_covered'].isna()
num_both_na_before_drop = both_na.sum()
if num_both_na_before_drop > 0:
    print(f"\nEliminando {num_both_na_before_drop} filas con NaN en ambas columnas...")
    df = df.loc[~both_na].reset_index(drop=True)
else:
    print("\nNo hay filas con NaN en ambas columnas para eliminar.")

# Resumen después de la limpieza
is_tot_na2 = df['surface_total'].isna()
is_cov_na2 = df['surface_covered'].isna()

print("\nDespués de rellenar y eliminar:")
print(f"  Filas totales: {len(df)}")
print(f"  NaN en surface_total: {is_tot_na2.sum()}")
print(f"  NaN en surface_covered: {is_cov_na2.sum()}")
print(f"  NaN en ambas columnas: {(is_tot_na2 & is_cov_na2).sum()}")

# Mostrar algunas filas restantes con problemas (si existen)
both_still_na = (is_tot_na2 & is_cov_na2)
if both_still_na.any():
    print('\nFilas con NaN en ambas columnas (deben revisarse o eliminarse manualmente):')
    display(df.loc[both_still_na, cols_show].head(10))
else:
    print('\nNo quedan filas con NaN en ambas columnas.')

Antes de rellenar:
  Filas totales: 37429
  NaN en surface_total: 4027
  NaN en surface_covered: 4182
  NaN en ambas columnas: 3950
  NaN solo en surface_total: 77
  NaN solo en surface_covered: 232

Ejemplos (NaN solo en surface_total):


,barrio,comuna,zona,precio,surface_total,surface_covered
481,San Cristobal,3,Centro/Oeste,70000.0,NaN,30.0
1049,Almagro,5,Centro/Oeste,150000.0,NaN,60.0
1495,Flores,7,Centro/Oeste,110000.0,NaN,43.0
1896,Flores,7,Centro/Oeste,115000.0,NaN,48.0
1992,Flores,7,Centro/Oeste,110000.0,NaN,43.0
2292,Nuñez,13,Norte,139000.0,NaN,60.0
2504,Nuñez,13,Norte,140000.0,NaN,40.0
2649,Nuñez,13,Norte,175000.0,NaN,52.0
3192,Almagro,5,Centro/Oeste,132000.0,NaN,1.0
3203,Almagro,5,Centro/Oeste,69000.0,NaN,33.0



Ejemplos (NaN solo en surface_covered):


,barrio,comuna,zona,precio,surface_total,surface_covered
100,San Cristobal,3,Centro/Oeste,47000.0,31.0,NaN
209,San Cristobal,3,Centro/Oeste,180000.0,57.0,NaN
472,San Cristobal,3,Centro/Oeste,47000.0,31.0,NaN
524,San Cristobal,3,Centro/Oeste,98000.0,58.0,NaN
736,Boedo,5,Centro/Oeste,143000.0,61.0,NaN
1026,Almagro,5,Centro/Oeste,100000.0,42.0,NaN
1214,Flores,7,Centro/Oeste,300000.0,120.0,NaN
1282,Flores,7,Centro/Oeste,300000.0,120.0,NaN
1297,Flores,7,Centro/Oeste,410000.0,180.0,NaN
1422,Flores,7,Centro/Oeste,125000.0,40.0,NaN



Ejemplos (NaN en ambas):


,barrio,comuna,zona,precio,surface_total,surface_covered
0,San Cristobal,3,Centro/Oeste,109403.0,NaN,NaN
1,San Cristobal,3,Centro/Oeste,47781.0,NaN,NaN
4,San Cristobal,3,Centro/Oeste,4144561.0,NaN,NaN
6,San Cristobal,3,Centro/Oeste,165307.0,NaN,NaN
7,San Cristobal,3,Centro/Oeste,138500.0,NaN,NaN
8,San Cristobal,3,Centro/Oeste,81400.0,NaN,NaN
9,San Cristobal,3,Centro/Oeste,81124.0,NaN,NaN
12,San Cristobal,3,Centro/Oeste,56000.0,NaN,NaN
14,San Cristobal,3,Centro/Oeste,83038.0,NaN,NaN
18,San Cristobal,3,Centro/Oeste,43405.0,NaN,NaN



Eliminando 3950 filas con NaN en ambas columnas...

Después de rellenar y eliminar:
  Filas totales: 33479
  NaN en surface_total: 0
  NaN en surface_covered: 0
  NaN en ambas columnas: 0

No quedan filas con NaN en ambas columnas.

Después de rellenar y eliminar:
  Filas totales: 33479
  NaN en surface_total: 0
  NaN en surface_covered: 0
  NaN en ambas columnas: 0

No quedan filas con NaN en ambas columnas.


## Pipeline XGBoost Final

In [130]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.base import clone
import warnings
from inspect import signature
from sklearn.feature_selection import VarianceThreshold

In [131]:

# Parámetros para habilitar GPU en XGBoost (si está disponible)
gpu_params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor'
}

## Pipelines de prepocesamiento

In [132]:
# Convertir columnas categóricas a tipo 'category'
# 'comuna' es un identificador categórico, no una variable numérica ordinal
categorical_cols = ['barrio', 'zona', 'comuna']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')
        print(f"Columna '{col}' convertida a category")

# Define el pipeline de preprocesamiento para las columnas numéricas
# Imputa los valores faltantes con la mediana y luego escala los datos.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define el pipeline de preprocesamiento para las columnas categóricas
# Imputa los faltantes con el valor más frecuente y luego aplica One-Hot Encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # handle_unknown evita errores si en test aparece una categoría nueva
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Crea el preprocesador maestro que combina los transformadores anteriores.
# Este se encarga de aplicar la transformación correcta a cada tipo de columna.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, selector(dtype_exclude="category")),
        ('cat', categorical_transformer, selector(dtype_include="category"))
    ],
    remainder='passthrough'
)

print("Pipeline de preprocesamiento creado exitosamente")
print(f"\nColumnas numéricas: {df.select_dtypes(exclude='category').columns.tolist()}")
print(f"Columnas categóricas: {df.select_dtypes(include='category').columns.tolist()}")




Columna 'barrio' convertida a category
Columna 'zona' convertida a category
Columna 'comuna' convertida a category
Pipeline de preprocesamiento creado exitosamente

Columnas numéricas: ['ambientes', 'habitaciones', 'baños', 'surface_total', 'surface_covered', 'precio', 'hab_x_norte', 'hab_x_sur', 'hab_x_centro_oeste', 'banos_x_norte', 'banos_x_sur', 'banos_x_centro_oeste', 'amb_x_norte', 'amb_x_sur', 'amb_x_centro_oeste', 'sup_cub_x_norte', 'sup_cub_x_sur', 'sup_cub_x_centro_oeste', 'sup_tot_x_norte', 'sup_tot_x_sur', 'sup_tot_x_centro_oeste', 'hab_x_comuna_14', 'hab_x_comuna_13', 'hab_x_comuna_2', 'hab_x_comuna_5', 'hab_x_comuna_15', 'hab_x_comuna_1', 'hab_x_comuna_6', 'hab_x_comuna_3', 'hab_x_comuna_12', 'hab_x_comuna_4', 'hab_x_comuna_7', 'hab_x_comuna_11', 'hab_x_comuna_10', 'hab_x_comuna_9', 'hab_x_comuna_8', 'banos_x_comuna_14', 'banos_x_comuna_13', 'banos_x_comuna_2', 'banos_x_comuna_5', 'banos_x_comuna_15', 'banos_x_comuna_1', 'banos_x_comuna_6', 'banos_x_comuna_3', 'banos_x_co

## Columna obejtivo

In [133]:
import math

def generate_exponential_bins(
    start_small_step=15_000,
    small_threshold=105_000,
    growth_factor=1.5,
    start_growth_from=105_000,
    growth_limit=1_000_000,
    large_step_after_growth=250_000
):
    """Genera una lista ordenada de límites (bin edges) siguiendo la regla descrita."""
    edges = [0]
    # small steps up to small_threshold
    current = 0
    while current < small_threshold:
        current += start_small_step
        edges.append(int(current))
    # growth phase
    bin_size = start_small_step
    current = edges[-1]
    while current < growth_limit:
        bin_size = max(start_small_step, int(bin_size * growth_factor))
        current += bin_size
        edges.append(int(current))
        # safety guard to avoid infinite loops
        if len(edges) > 1000:
            break
    # after growth_limit, fixed large steps
    current = edges[-1]
    while current < growth_limit + 10 * large_step_after_growth:  # crear varias hasta pasar el límite
        current += large_step_after_growth
        edges.append(int(current))
        if len(edges) > 2000:
            break
    return sorted(list(dict.fromkeys(edges)))  # unicidad y orden

# pre-generate edges for reproducibilidad
BIN_EDGES = generate_exponential_bins()

# función para mapear precio a etiqueta lower-upper basada en BIN_EDGES
def map_price_to_exp_bin(p):
    if pd.isna(p):
        return np.nan
    try:
        p = float(p)
    except Exception:
        return np.nan
    if p < 0:
        return np.nan
    # encontrar el primer edge > p
    for i in range(1, len(BIN_EDGES)):
        lower = BIN_EDGES[i-1]
        upper = BIN_EDGES[i]
        if lower <= p < upper:
            return f"{int(lower)}-{int(upper)}"
    # si p >= último edge, agrupar en bins de large_step_after_growth a partir del último edge
    last = BIN_EDGES[-1]
    step = 250_000
    offset = int((p - last) // step) * step
    lower = last + offset
    upper = lower + step
    return f"{int(lower)}-{int(upper)}"

# Crear y asignar etiquetas binned usando la estrategia exponencial
y_precio = df['precio']
y_binned = y_precio.map(map_price_to_exp_bin)

In [134]:
# Filtrar bins de precio con menos de 5 muestras
# Esto evita problemas con clases raras al hacer train/test split

# Contar cuántas muestras tiene cada bin
bin_counts = y_binned.value_counts()
print(f"Total de bins únicos: {len(bin_counts)}")
print(f"Bins con <5 muestras: {(bin_counts < 5).sum()}")

# Identificar bins válidos (con al menos 5 muestras)
min_samples = 5
valid_bins = bin_counts[bin_counts >= min_samples].index

# Filtrar el dataframe para mantener solo filas con bins válidos
mask_valid = y_binned.isin(valid_bins)
n_removed = (~mask_valid).sum()

print(f"\nFilas a eliminar por bins raros: {n_removed} ({100*n_removed/len(df):.2f}%)")

if n_removed > 0:
    df = df.loc[mask_valid].reset_index(drop=True)
    print(f"Nuevo tamaño del dataset: {len(df)} filas")
else:
    print("No se eliminaron filas.")

# Actualizar y_binned y y_precio también
y_binned = y_binned.loc[mask_valid].reset_index(drop=True)
y_precio = y_precio.loc[mask_valid].reset_index(drop=True)

# Verificar distribución final
final_counts = y_binned.value_counts()
print(f"\nBins finales: {len(final_counts)}")
print(f"Min muestras por bin: {final_counts.min()}")
print(f"Max muestras por bin: {final_counts.max()}")
print(f"Mediana muestras por bin: {final_counts.median()}")

Total de bins únicos: 34
Bins con <5 muestras: 7

Filas a eliminar por bins raros: 13 (0.04%)
Nuevo tamaño del dataset: 33466 filas

Bins finales: 27
Min muestras por bin: 5
Max muestras por bin: 5725
Mediana muestras por bin: 142.0


## Datos de entrenamiento

In [140]:
# Crear X_final y Y_final para probar el modelo
# - Usamos `df` cargado previamente (ya filtrado por bins válidos)
# - Y_final será y_binned (los bins de precio ya filtrados)
# - X_final será el DataFrame sin la columna precio

if 'df' not in globals():
    raise NameError('No se encuentra el DataFrame `df`. Ejecuta la celda de carga antes de correr esto.')

if 'y_binned' not in globals():
    raise NameError('No se encuentra `y_binned`. Ejecuta la celda de creación de bins antes de correr esto.')

# Usar y_binned como target (los bins filtrados)
Y_final = y_binned.copy()

# Crear X_final: todo menos la columna precio
X_final = df.drop(columns=['precio'])

# Asegurar que 'precio_numeric' no está en X_final
if 'precio_numeric' in X_final.columns:
    X_final = X_final.drop(columns=['precio_numeric'])

# Información rápida
print('Target: y_binned (bins de precio)')
print('X_final shape:', X_final.shape)
print('Y_final shape:', Y_final.shape)
print(f'Clases únicas en Y_final: {Y_final.nunique()}')

# Mostrar primeras filas para inspección
display(X_final.head())

Target: y_binned (bins de precio)
X_final shape: (33466, 338)
Y_final shape: (33466,)
Clases únicas en Y_final: 27


,barrio,ambientes,habitaciones,baños,surface_total,surface_covered,comuna,zona,hab_x_norte,hab_x_sur,...,sup_tot_x_barrio_villa_ortuzar,sup_tot_x_barrio_parque_chas,sup_tot_x_barrio_pompeya,sup_tot_x_barrio_versalles,sup_tot_x_barrio_velez_sarsfield,sup_tot_x_barrio_parque_avellaneda,sup_tot_x_barrio_agronomía,sup_tot_x_barrio_villa_real,sup_tot_x_barrio_villa_soldati,sup_tot_x_barrio_villa_riachuelo
0,San Cristobal,2.0,1.0,1.0,42.0,40.0,3,Centro/Oeste,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,San Cristobal,2.0,3.0,1.0,45.0,41.0,3,Centro/Oeste,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,San Cristobal,3.0,2.0,1.0,78.0,70.0,3,Centro/Oeste,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,San Cristobal,3.0,2.0,1.0,60.0,57.0,3,Centro/Oeste,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,San Cristobal,3.0,2.0,1.0,58.0,54.0,3,Centro/Oeste,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
# XGBoost necesita etiquetas numéricas
le_xgb = LabelEncoder()
y_xgb_enc = le_xgb.fit_transform(Y_final)

In [142]:
pipeline_xgb1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        n_estimators=250,
        max_depth=10,            # Árboles muy profundos para capturar interacciones complejas
        learning_rate=0.05,      # Un learning rate un poco más bajo para compensar la profundidad
        objective='multi:softmax',
        num_class=len(le_xgb.classes_),
        subsample=0.75,
        colsample_bytree=0.6,    # Usa solo el 60% de las features por árbol
        gamma=0.2,               # Más agresivo en la poda de ramas
        random_state=42,
        n_jobs=-1,
        use_label_encoder=False,
        eval_metric='mlogloss',
        **gpu_params
    ))
])

## Entrenamiento y evaluacion del Pipeline: pipeline_xgb1 

In [143]:
X_train_xgb, X_val_xgb, y_train_xgb, y_val_xgb = train_test_split(
    X_final,
    y_xgb_enc,
    test_size=0.2,
    random_state=42
)

In [144]:
# --- EVALUAR Y GUARDAR pipeline_xgb1 ---
import os
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.base import clone

print("Entrenamiento completo para pipeline_xgb1.")

# Entrenamiento en train/validation split para métricas
pipeline_xgb1_train = clone(pipeline_xgb1)
pipeline_xgb1_train.fit(X_train_xgb, y_train_xgb)

# Métricas sobre el conjunto de entrenamiento
y_train_pred_xgb1 = pipeline_xgb1_train.predict(X_train_xgb)
train_accuracy_xgb1 = accuracy_score(y_train_xgb, y_train_pred_xgb1)
print(f"\nAccuracy (train): {train_accuracy_xgb1:.4f}")

y_train_true_labels = le_xgb.inverse_transform(y_train_xgb)
y_train_pred_labels_xgb1 = le_xgb.inverse_transform(y_train_pred_xgb1)
print("\nReporte de clasificación (train):")
print(classification_report(y_train_true_labels, y_train_pred_labels_xgb1, zero_division=0))
print("Matriz de confusión (train):")
print(confusion_matrix(y_train_true_labels, y_train_pred_labels_xgb1))

# Métricas sobre el conjunto de validación
y_val_pred_xgb1 = pipeline_xgb1_train.predict(X_val_xgb)
valid_accuracy_xgb1 = accuracy_score(y_val_xgb, y_val_pred_xgb1)
print(f"\nAccuracy (valid): {valid_accuracy_xgb1:.4f}")

y_val_true_labels = le_xgb.inverse_transform(y_val_xgb)
y_val_pred_labels_xgb1 = le_xgb.inverse_transform(y_val_pred_xgb1)
print("\nReporte de clasificación (valid):")
print(classification_report(y_val_true_labels, y_val_pred_labels_xgb1, zero_division=0))
print("Matriz de confusión (valid):")
print(confusion_matrix(y_val_true_labels, y_val_pred_labels_xgb1))

# Reentrenar con todo el dataset
pipeline_xgb1_full = clone(pipeline_xgb1)
pipeline_xgb1_full.fit(X_final, y_xgb_enc)
print("\nPipeline final (xgb1) reentrenado con todo el dataset.")

modelo_y_encoder_xgb1 = {
    'pipeline': pipeline_xgb1_full,
    'encoder': le_xgb
}

carpeta_destino = 'C:\\Users\\cirri\\OneDrive\\Desktop\\Cienciadedatos\\BairesProp\\model'
os.makedirs(carpeta_destino, exist_ok=True)
ruta_xgb1 = os.path.join(carpeta_destino, 'modelo_clasificador_precios_xgb1V3.pkl')

try:
    joblib.dump(modelo_y_encoder_xgb1, ruta_xgb1)
    print(f"\n¡Modelo y encoder (xgb1) guardados en: {ruta_xgb1}!")
except Exception as e:
    print(f"Error al guardar xgb1: {e}")

Entrenamiento completo para pipeline_xgb1.


c:\Users\cirri\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:22:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cirri\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:22:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cirri\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:2676: UserWarning: [13:23:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` p


Accuracy (train): 0.7395

Reporte de clasificación (train):
                 precision    recall  f1-score   support

        0-15000       1.00      0.50      0.67         8
  105000-127500       0.64      0.66      0.65      3272
1213286-1463286       0.95      0.89      0.92       157
  127500-161250       0.69      0.77      0.72      4612
1463286-1713286       0.95      0.93      0.94       113
    15000-30000       0.58      0.54      0.56        13
  161250-211875       0.76      0.75      0.75      4446
1713286-1963286       0.90      0.93      0.92        41
1963286-2213286       0.93      0.95      0.94        39
  211875-287812       0.80      0.79      0.80      3948
2213286-2463286       1.00      0.60      0.75        20
2463286-2713286       0.76      0.88      0.82        33
2713286-2963286       1.00      0.83      0.91         6
  287812-401717       0.82      0.81      0.81      2619
2963286-3213286       0.78      0.95      0.86        19
    30000-45000       0.91

c:\Users\cirri\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:23:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\cirri\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:23:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Pipeline final (xgb1) reentrenado con todo el dataset.

¡Modelo y encoder (xgb1) guardados en: C:\Users\cirri\OneDrive\Desktop\Cienciadedatos\BairesProp\model\modelo_clasificador_precios_xgb1V3.pkl!

¡Modelo y encoder (xgb1) guardados en: C:\Users\cirri\OneDrive\Desktop\Cienciadedatos\BairesProp\model\modelo_clasificador_precios_xgb1V3.pkl!


c:\Users\cirri\AppData\Local\Programs\Python\Python313\Lib\pickle.py:576: UserWarning: [13:24:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  rv = reduce(self.proto)
